In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pydub import AudioSegment

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MaxAbsScaler

In [2]:
def transform_audios(value, quantidade_arquivos, pasta, info):

    lista_arquivos = []

    lista_audios = []

    lista_classe = []

    for palavra in info:

        for i in range(quantidade_arquivos):
    
            string = f"{pasta}/{0}{i+1}_{palavra}.mp3"
            lista_arquivos.append(string)

            if palavra == "abrir":
                lista_classe.append(0)

            elif palavra == "compilar":
                lista_classe.append(1)
            
            elif palavra == "executar":
                lista_classe.append(2)

            else:
                lista_classe.append(3)

    for arquivo in lista_arquivos:
    
        sound = AudioSegment.from_mp3(arquivo)
        raw_data = sound._data

        audio = []
        
        for si in range(len(raw_data)//2):
            sample = int.from_bytes(raw_data[si * 2 : si * 2 + 2], 'little', signed=True)
            if sample < -30 or sample > 30:
                audio.append(sample)
        
        audio = np.array(audio)
        
        shape = audio.shape
  
        calculo = int(shape[0]) / value

        calculo = int(calculo)
        
        audio = audio[::calculo]
        
        audio = audio[:value]
        
        lista_audios.append(audio)

    df = pd.DataFrame(lista_audios)
    df["classe"] = lista_classe

    return df

In [14]:
resultado = []

value = 75

fim = False

info = ["abrir", "compilar", "executar", "testar"]

while fim == False:

    try:
        
        quantidade_arquivos_por_classe = 8

        pasta = "audios_treinamento"

        arquivos_audio_treinamento = transform_audios(value, quantidade_arquivos_por_classe, pasta, info)

        x = arquivos_audio_treinamento.drop(['classe'], axis= 1)
        y = arquivos_audio_treinamento["classe"]

        scaler = MaxAbsScaler().fit(x)  

        x = scaler.transform(x)

        quantidade_arquivos_por_classe = 2
        pasta = "audios_teste"

        arquivos_audio_teste = transform_audios(value, quantidade_arquivos_por_classe, pasta, info)

        x_test = arquivos_audio_teste.drop(['classe'], axis= 1)

        y_test = arquivos_audio_teste["classe"] 

        scaler = MaxAbsScaler().fit(x_test)

        x_test = scaler.transform(x_test)

        knn_model = KNeighborsClassifier(n_neighbors=3)
        knn_model.fit(x, y)

        y_pred = knn_model.predict(x_test)

        score = knn_model.score(x_test, y_test)

        resultado.append({"value": value, "score": score})

        value += 150

    except:
        fim = True

In [ ]:
resultado